In [1]:
from fairseq import (
    checkpoint_utils, distributed_utils, metrics, options, progress_bar, tasks, utils
)
from fairseq.data import iterators  
from fairseq.trainer import Trainer
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import torch

/home/tnguyen/dialogue-text-summarization-dokument/.venv/lib/python3.6/site-packages/torch/torch_version.py:3: UserWarning: Module fairseq was already imported from /home/tnguyen/dialogue-text-summarization-dokument/Multi-View-Seq2Seq/fairseq_multi_view/fairseq/__init__.py, but /home/tnguyen/dialogue-text-summarization-dokument/Multi-View-Seq2Seq/train_sh is being added to sys.path
  from pkg_resources import packaging  # type: ignore[attr-defined]


In [2]:
class Args():
    no_progress_bar = False
    log_interval = 1000
    log_format = None
    tensorboard_logdir = ""
    seed = 14632
    cpu = True
    fp16 = False
    memory_efficient_fp16 = False
    fp16_no_flatten_grads = False
    fp16_init_scale = 128
    fp16_scale_window = None
    fp16_scale_tolerance = 0.0
    min_loss_scale = 0.0001
    threshold_loss_scale = None
    user_dir = None
    empty_cache_freq = 0
    all_gather_list_size = 16384
    multi_views = True
    balance = True
    lr_weight = 1000.0
    T = 0.2
    criterion = "label_smoothed_cross_entropy"
    tokenizer = None
    bpe = None
    optimizer = "adam"
    lr_scheduler = "polynomial_decay"
    task = "translation"
    num_workers = 1
    skip_invalid_size_inputs_valid_test = True
    max_tokens = 718
    max_sentences = None
    required_batch_size_multiple = 1
    dataset_impl = None
    train_subset = "train"
    valid_subset = "valid"
    validate_interval = 1
    fixed_validation_seed = None
    disable_validation = False
    max_tokens_valid = 718
    max_sentences_valid = None
    curriculum = 0
    distributed_world_size = 1
    distributed_rank = 0
    distributed_backend = "nccl"
    distributed_init_method = None
    distributed_port = -1
    device_id = 0
    distributed_no_spawn = False
    ddp_backend = "no_c10d"
    bucket_cap_mb = 25
    fix_batches_to_gpus = False
    find_unused_parameters = True
    fast_stat_sync = False
    broadcast_buffers = False
    arch = "bart_large"
    max_epoch = 0
    max_update = 0
    clip_norm = 0.1
    sentence_avg = False
    update_freq = [32]
    lr = [3e-05]
    min_lr = -1
    use_bmuf = False
    save_dir = "checkpoints"
    restore_file = "../checkpoints_multi_view/bart.large/model.pt"#"/home/tnguyen/Multi-View-Seq2Seq/checkpoints_multi_view/bart.large/model.pt"
    reset_dataloader = True
    reset_lr_scheduler = False
    reset_meters = True
    reset_optimizer = True
    optimizer_overrides = "{}"
    save_interval = 1
    save_interval_updates = 0
    keep_interval_updates = -1
    keep_last_epochs = -1
    keep_best_checkpoints = -1
    no_save = False
    no_epoch_checkpoints = True
    no_last_checkpoints = False
    no_save_optimizer_state = False
    best_checkpoint_metric = "loss"
    maximize_best_checkpoint_metric = False
    patience = -1
    no_token_positional_embeddings = False
    no_cross_attention = False
    cross_self_attention = False
    layer_wise_attention = False
    encoder_layerdrop = 0
    decoder_layerdrop = 0
    encoder_layers_to_keep = None
    decoder_layers_to_keep = None
    label_smoothing = 0.1
    adam_betas = "(0.9, 0.999)"
    adam_eps = 1e-08
    weight_decay = 0.01
    use_old_adam = False
    force_anneal = None
    warmup_updates = 200
    end_learning_rate = 0.0
    power = 1.0
    total_num_update = 5000
    data = "dialogsum-bin_2"
    source_lang = "source"
    target_lang = "target"
    load_alignments = False
    left_pad_source = "True"
    left_pad_target = "False"
    max_source_positions = 1024
    max_target_positions = 1024
    upsample_primary = 1
    truncate_source = True
    eval_bleu = False
    eval_bleu_detok = "space"
    eval_bleu_detok_args = None
    eval_tokenized_bleu = False
    eval_bleu_remove_bpe = None
    eval_bleu_args = None
    eval_bleu_print_samples = False
    layernorm_embedding = True
    share_all_embeddings = True
    share_decoder_input_output_embed = True
    dropout = 0.1
    attention_dropout = 0.1
    encoder_embed_path = None
    encoder_embed_dim = 1024
    encoder_ffn_embed_dim = 4096
    encoder_layers = 12
    encoder_attention_heads = 16
    encoder_normalize_before = False
    encoder_learned_pos = True
    decoder_embed_path = None
    decoder_embed_dim = 1024
    decoder_ffn_embed_dim = 4096
    decoder_layers = 12
    decoder_attention_heads = 16
    decoder_normalize_before = False
    decoder_learned_pos = True
    relu_dropout = 0.0
    adaptive_softmax_cutoff = None
    adaptive_softmax_dropout = 0
    decoder_output_dim = 1024
    decoder_input_dim = 1024
    no_scale_embedding = True
    activation_fn = "gelu"
    pooler_activation_fn = "tanh"
    pooler_dropout = 0.0

    
args = Args()
task = tasks.setup_task(args)

In [3]:
model = task.build_model(args)
criterion = task.build_criterion(args)

In [4]:
model

BARTModel(
  (encoder): TransformerEncoder(
    (embed_tokens): Embedding(50264, 1024, padding_idx=1)
    (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (

In [ ]:
#77644.94844341278 s

In [6]:
trainer = Trainer(args, task, model, criterion)
extra_state, epoch_itr = checkpoint_utils.load_checkpoint(args, trainer)

46 samples have invalid sizes and will be skipped, max_positions=(718, 718), first few sample ids=[6174, 7632, 8680, 6939, 8149, 5984, 9284, 6787, 8041, 8713]


group1: 
511
group2: 
12
here schedule!
!!! 12460 12460


In [7]:
args.update_freq

[32]

In [25]:
epoch = 1
itr = epoch_itr.next_epoch_itr(
    fix_batches_to_gpus=args.fix_batches_to_gpus,
    shuffle=(epoch_itr.epoch >= args.curriculum),
)
update_freq = (
    args.update_freq[-1]
)
g_itr = iterators.GroupedIterator(itr, update_freq)

In [9]:
progress = progress_bar.build_progress_bar(
    args, g_itr, epoch, no_progress_bar='simple'
)

In [10]:
task.begin_epoch(epoch_itr.epoch, trainer.get_model())

In [26]:
itr

In [11]:
import math
valid_subsets = args.valid_subset.split(',')
max_update = args.max_update or math.inf

In [32]:
tmp = next(iter(epoch_itr))

TypeError: 'EpochBatchIterator' object is not iterable

In [30]:
tmp.keys()

dict_keys(['id', 'nsentences', 'ntokens', 'net_input', 'target'])

In [35]:
ds = epoch_itr.dataset

In [38]:
ds[0].keys()

dict_keys(['id', 'source', 'source2', 'target'])